# **Week 1 : Linear Algebra and Calculus**

In this assignment, we shall explore the concepts of analytic and numeric computation of gradients. Further, we will have a look at the computation graph which is a central concept to building a neural network. For learning how gradients are computed analytically, refer to the resources provided in this week.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWsYeD8ZlYxFBB33qIn7bwQvP-KuvLZXJOoA&usqp=CAU"
 style="float:center;width:50px;height:50px;">

# **Importing Libraries**
Feel free to import any additional libraries required

In [ ]:
import numpy as np
import random
import math

# *Problem 1(a)*

In this problem, we shall be exploring the concepts of analytic and numeric computation of gradients for scalar valued functions. 

\begin{equation}
z = w^{T}x + b \\ 
\hat{y} = \sigma(z) = \frac{1}{1+e^{-z}}\\ 
L(\hat{y}, y) = y.log(\hat{y}) + (1-y).log(1-\hat{y})
\end{equation}

For this set of equations, the vector w, and scalars b, y are to be treated as constants. \\

Now, let us find the analytic gradient of the function L with respect to the function x. That is, compute $\frac{\delta L}{\delta x}$. Write the answer obtained as code in the function provided.

Initialise $w$ to a $10 \times 1$ vector sampled over a standard multivariate gaussian distribution, b to a uniform random variable over the interval $(0, 1)$, y to a bernoulli random variable over $\{0, 1\}$

In [ ]:
w = np.random.normal(size=(10, 1))
b = random.random()
y = random.choice([0,1])

In [ ]:
def analytic_grad(x) : 

    z = np.sum(np.dot(w.T, x)) + b
    k = 1/(1+math.exp(-z))
    dz = np.sum(w)
    result = dz*(y-k)
    return result

x = np.random.normal(size=(10,1))

# *Problem 1(b)*

Now, we compute the numeric gradient for the function L. Refer to [this](https://stackoverflow.com/questions/38854363/is-there-any-standard-way-to-calculate-the-numerical-gradient) stack exchange post to see how numeric gradients are computed

In [ ]:
def numeric_grad(x) : 

    z = lambda x : np.sum(np.matmul(w.T, x)) + b
    _y = lambda z : 1/(1+math.exp(-z))
    L = lambda _y :  y*math.log(_y, math.e) + (1-y)*math.log(1-_y, math.e)
    eps = math.exp(-15)
    
    num_grad = (L(_y(z(x+eps))) - L(_y(z(x))))/(eps)
    return num_grad


# *Problem 2*

Here, we'll be looking at computational graphs, and their calculus, finding gradients of  variables with respect to other variables in the graph.



We'll be looking at nodes of the graph that are operation based, i.e., each operation performed has a node associated with it.

We'll provide you with example implementations for three of the nodes, the *add* node, the *nth power* node and the *sine* node, you'll have to write the classes for all the other nodes.

# **Multi-input nodes**

In [ ]:
class Add: 
  
  # A class to add multiple variables together
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # Scalar addition is taken care of in a separate node, we could have fit into this node but thought it might be easier if it wasn't.
                               # So cases like "a + 1" are to be done separately, and cases like "b + c + d + 4" can be done as "(b + c + d) + 4" or "b + c + (d + 4)", since our scalar addition supports only one variable and one scalar, we'll get to that later
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    return np.sum(self.lst_values) # This computes the sum of all the variables
  
  def compute_gradients(self):
    return dict(zip(self.lst_names, np.ones(len(self.lst_names)))) # This gives the gradient of the sum wrt to all the input variables, as a dictionary, will be used later


class Multiply: 
  
  #Everything's almost the same as the add class, but this deals with multiplication of more than 1 variables
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # Scalar multiplication is taken care of in a separate node
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Code to compute the output of this operation
    return np.prod(self.lst_values)
  
  def compute_gradients(self):
    # Code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    arr_prod = [self.compute_output() for i in range(len(self.lst_values))]
    arr_prod = np.divide(arr_prod, self.lst_values)
    return dict(zip(self.lst_names, arr_prod))


# **Scalar multiplication/addition nodes**

In [ ]:
class Add_Scalar: 
  
  # A class to add a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, and the second should be a string of the scalar value to be added.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    # Code to compute the output of this operation
    return np.sum(self.lst_values)

  def compute_gradients(self):
    # Code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    try:
      int(self.lst_names[1])
      return dict(zip(self.lst_names, [1, 0]))
    except:
      return dict(zip(self.lst_names, [0, 1]))


class Multiply_Scalar: 
  
  # A class to multiply a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, and the second should be a string of the scalar value to be multiplied.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Code to compute the output of this operation
    return np.prod(self.lst_values)

  def compute_gradients(self):
    # Code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    try:
      int(self.lst_names[1])
      return dict(zip(self.lst_names, [self.lst_names[1], 0]))
    except:
      return dict(zip(self.lst_names, [0, self.lst_names[0]]))


# **Nodes for special functions**

In [ ]:
class Power:

  def __init__(self, name, value, exponent):
    self.name = name # Name of the variable to be exponentiated
    self.value = value # Value of the variable
    self.exponent = exponent # Value of the exponent
  
  def compute_output(self):
    return np.power(self.value, self.exponent)
  
  def compute_gradients(self):
    return {self.name : self.exponent*(np.power(self.value, (self.exponent - 1)))}

class Sine: 

  # A class to apply the sine function on a variable
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    return np.sin(self.value)
  
  def compute_gradients(self):
    return {self.name : np.cos(self.value)}


class Logarithm:

  # A class to compute the logarithm of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    return np.log(self.value)

  def compute_gradients(self):
    return {self.name : 1/(self.value)}

class Exponential: 

  # A class to compute the exponential of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    return np.exp(self.value)

  def compute_gradients(self):
    return {self.name : np.exp(self.value)}


Now that we have these classes, let's use them to actually find gradients of complex networks. We finally bring in the idea of a computational graph, which makes it much easier for the gradients to be computed.

This is the image of the example graph that we want you to work with. 





<div>
<img src="https://drive.google.com/uc?id=1VtI1lf85bG8cO1u_8l0D0rqVGhwHQtPK"
 width="500" height="500">
</div>

\begin{equation}
d = 3a \\ 
e = abc \\ 
f = sin(c) \\ 
g = exp(e) \\ 
h = a + d + g + f
\end{equation}

# **Forward Propogation**

In [ ]:
def forward_prop(a, b, c) : 
    '''
    Input : real numbers a, b, c.

    Outputs : A dictionary of the values at each node with keys as the names of nodes
    Grads : A dictionary of the gradients at each edge with keys as a pair of nodes 
    
    e.g. Grads["ce"] = ...
    '''
    # Output['e'] := e(a, b, c) = abc
    # Declaring an empty Dictionary
    Outputs = {}
    Outputs['d'] = Multiply_Scalar(['3', 'a'], [3, a]).compute_output()
    Outputs['e'] = Multiply(['a', 'b', 'c'], [a, b, c]).compute_output()
    Outputs['f'] = Sine('c', c).compute_output()
    Outputs['g'] = Exponential('e', Outputs['e']).compute_output()
    Outputs['h'] = Add(['a', 'd', 'g', 'f'], [a, Outputs['d'], Outputs['g'], Outputs['f']]).compute_output()
    
    # Grads['ah'] := ∂h/∂a
    # Declaring an empty Dictionary
    Grads = {}
    Grads['ad'] = Multiply_Scalar(['3', 'a'], [3, a]).compute_gradients()['a']
    Grads['ae'] = Multiply(['a' , 'b', 'c'], [a, b, c]).compute_gradients()['a']
    Grads['be'] = Multiply(['a' , 'b', 'c'], [a, b, c]).compute_gradients()['b']
    Grads['ce'] = Multiply(['a' , 'b', 'c'], [a, b, c]).compute_gradients()['c']
    Grads['cf'] = Sine('c', c).compute_gradients()['c']
    Grads['eg'] = Exponential('e', Outputs['e']).compute_gradients()['e']
    Grads['dh'] = Add(['a', 'd', 'g', 'f'], [a, Outputs['d'], Outputs['g'], Outputs['f']]).compute_gradients()['d']
    Grads['fh'] = Add(['a', 'd', 'g', 'f'], [a, Outputs['d'], Outputs['g'], Outputs['f']]).compute_gradients()['f']
    Grads['gh'] = Add(['a', 'd', 'g', 'f'], [a, Outputs['d'], Outputs['g'], Outputs['f']]).compute_gradients()['g']
    
    return (Outputs, Grads)


# **Backward Propogation**

Most of the usage of computational graphs in Machine Learning centers around finding the gradients of a particular loss, wrt to all the parameters. Here, your task is to do a simpler version of that.

Use the classes you wrote to calculate the following gradients : 


*   $ \frac{\partial h}{\partial a}$
*   $ \frac{\partial h}{\partial b}$
*   $ \frac{\partial h}{\partial c}$

Use the technique of *backpropogation* to code out the gradients step-wise, along all possible chains of the graph starting from $h$ and ending at $a,b,c$ respectively. 

Don't try to directly get these values without backpropogation, it might be easier here, but with complicated neural networks it wouldn't be :) 



In [ ]:
def backward_prop(a, b, c, Outputs, Grads) : 
    '''
    Input : a, b, c (input layer); Outputs (values at each node); Grads (gradients stored at each edge)
    Retuns : result (gradients w.r.t a, b, c)
    '''
    # Computing gradients using pre-calculated gradients and chain rule
    Grads['ah'] = float(Grads['dh']+1)*float(Grads['ad'])
    Grads['ch'] = Grads['gh']*Grads['eg']*Grads['ce'] + Grads['fh']*Grads['cf']
    Grads['bh'] = Grads['gh']*Grads['eg']*Grads['be']

    # returning a list with size 3
    result = [Grads['ah'], Grads['bh'], Grads['ch']]
    return result

# **Combining both processes**

For the purpose of values, assume that $a = 3, b = 1, c = 2$. Here, we call both forward and backward propogation functions to demonstrate functionality of the functions above.

In [ ]:
# Initialisation
a = 3
b = 1
c = 2

# Forward propogation
(Outputs, Grads) = forward_prop(a, b, c)

print(f"Value obtained upon forward propogation : { Outputs['h'] }")

# Backward propogation
result = backward_prop(a, b, c, Outputs, Grads)
print(f'Values of gradients are : {result[0]}, {result[1]}, {result[2]}')